<a href="https://colab.research.google.com/github/sflin/SchneewittchensStiefmutter/blob/master/SchneewittchensStiefmutter.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# **Schneewittchens Stiefmutter** - Ein Tool zum Generieren von Fragen aus Texten

Dies ist ein Google Colab Notebook, das alle nötigen Informationen enthält um sich automatisch Fragen aus Texten zu generieren. Das Notebook ist so konzipiert, dass es auch ohne Programmierkenntnisse problemlos verwendbar ist.

Bitte folgen Sie den Anweisungen in den Kästchen und führen Sie diese Schritt für Schritt **der Reihe nach aus**.

> **Wichtig**: Ein Notebook besteht aus verschiedenen Code-Zellen (Kästchen). Um eine Zelle auszuführen, **klickt man zuerst in die Zelle *hinein*** und **dann auf das grau hinterlegte Dreieck** in der linken Spalte, so wie im  Screenshot: ![Dreieck](https://drive.google.com/uc?id=1AdQCZGnFTr4twrSd7xSYPICPELhcKXfi)

---


This is a Google Colab Notebook containing all necessary information to generate questions out of German texts. The notebook was prepared in such a way that it should be executable also by those who are not familiar with programming. 

Please follow the specifications in the cells and **execute them step by step**.

> **Important**: A notebook consists of different code-cells. In order to run a cell, **click first *into* the cell** and then **click the grey triangle** on the left of the cell (see screenshot above).

**Hilfe**

Falls ein Fehler auftritt, überprüfen Sie bitte die folgenden Punkte:
1. Haben Sie alle Kästchen **in der beschriebenen Reihenfolge** ausgeführt?
2. Fehler beim Herunterladen der generierten Fragen: Bestätigen Sie, dass das Notebook mehrere Dateien herunterladen darf (es wird ein Pop-Up oder Balken oder eine entsprechende Meldung im Browser angezeigt). 
3. Versuchen Sie das Kästchen, in dem der Fehler auftritt, nochmals auszuführen (ins Kästchen klicken und CTRL + F10 drücken).
3. Versuchen Sie das Tool mit dem Chrome-Browser auszuführen.
4. Melden Sie sich unter selin.fabel@uzh.ch

---

**Troubleshooting**
If an error occurs, please check the following points:
1. Have you run all cells **in the order described**?
2. Error while downloading the generated questions: confirm that the notebook is allowed to download multiple files (a pop-up or horizontal bar at the top or a corresponding message will be displayed in the browser) 
3. Try to re-run the cell where the error occurs (click in the box and hit CTRL + F10)
3. Try to execute the tool with the Chrome-browser.
4. Write an email to to selin.fabel@uzh.ch

# Tool

In [0]:
#@title Spezifizieren Sie die folgende URL um Text von einer Webseite zu laden und führen Sie die Code-Zelle aus: { run: "auto" }
#@markdown **Hinweis**: Die Extraktion wurde mit Texten im Format von Wikipedia-Artikeln implementiert und getestet. Für das Parsen von Non-Wikipedia-Artikel empfiehlt es sich, den Textfeld-Input oder den Dateiinput zu verwenden (s. nächsten Zellen).

#@markdown Specify the following URL to load text from an existing website and run the cell:\
#@markdown **Hint**: The extraction was implemented and tested with texts in the format of Wikipedia articles. For parsing of non-Wikipedia articles it is recommended to use the text field input or file input (see cells below).
URL = 'https://de.wikipedia.org/wiki/Emmy_von_Rhoden' #@param {type:"string"}
content = None

In [0]:
#@title ...oder führen Sie diese Zelle aus um Text aus einer Datei hochzuladen.
#@markdown ...or run this code cell to upload text from a file. 
from google.colab import files
import io
content = files.upload()
content = io.BytesIO(list(content.values())[0]).read().decode('UTF-8') 
URL = None

In [0]:
#@title ...oder kopieren Sie ihren Text in dieses Feld. { run: "auto" }
#@markdown ...or paste text in this field.

content = "" #@param {type:"string"}
URL = None

In [0]:
#@title Falls es sich im Text um eine Person dreht, sollen Koreferenzen (z.B. "sie" oder "er") ersetzt werden? Bitte kreuzen Sie die Box "replace_objects" an, wählen das Geschlecht aus dem "gender_dropdown" aus und geben den Ersatz im Feld "replacement" an. Dann führen Sie diese Zelle aus. { run: "auto" }
#@markdown If the text is about a person, should co-referent items (e.g. "sie" (she) or "er" (he)) be replaced? Please check the box "replace_objects", select the gender from the "gender_dropdown" and fill the "replacement". Then execute this cell.  
replace_objects = True #@param {type:"boolean"}
gender_dropdown = "female" #@param ["male", "female", "neutral"]
replacement = 'Emmy von Rhoden' #@param {type:"string"}



In [0]:
#@title Sollen die Fragen danach heruntergeladen werden? { run: "auto" }
#@markdown Would you like to download the generated questions?  
download_questions = False #@param {type:"boolean"}

In [0]:
#@title -- { display-mode: "form" }
#@markdown ## ![Start](https://drive.google.com/uc?id=1aUu3b7eGdXOb2-wB3TirOL0dDQQorLXz) &nbsp; Nur bei der **ersten** Ausführung (sonst weiter bei "Re-Start"-Zelle weiter unten): Zelle anwählen und CTRL+F10 drücken. 

#@markdown **First execution only** (otherwise see "Re-Start"-cell below): Select this cell. Then hit CTRL+F10.

In [0]:
#@title Installation von benötigten Modulen, Modell herunterladen (dauert einen Moment) { display-mode: "form" }
#@markdown Install required packages, download model (this may take a while)
!pip install dateparser
!pip3 install duden
!pip install ktrain==0.11.0
!pip install -U textblob-de
!sudo locale-gen de_DE.utf8
!update-locale LANG=de_DE.UTF-8
!wget https://dl.fbaipublicfiles.com/fasttext/vectors-crawl/cc.de.300.bin.gz
!gunzip -k cc.de.300.bin.gz
!wget https://www.dropbox.com/s/fqcfqxuq1zjvd6x/predictor-crf.preproc?dl=0 -O predictor-crf.preproc
!wget https://www.dropbox.com/s/hxkf4xx8d7v01hn/predictor-crf?dl=0 -O predictor-crf
!wget https://www.dropbox.com/s/q52fx19dam6uujz/df-types.parquet.gzip?dl=0 -O df-types.parquet.gzip
from IPython.display import clear_output
clear_output()

In [0]:
#@title Imports tätigen { display-mode: "form" }
#@markdown Organize imports
%tensorflow_version 2.x
import dateparser
import datetime
import duden
import gensim
import itertools
import ktrain
import numpy as np
import pandas as pd
import random
import re
import requests
import spacy
import spacy.cli
import tensorflow as tf
from bs4 import BeautifulSoup
from dateutil.parser import parse
from dateutil.parser import parserinfo
from gensim.models.wrappers import FastText
from google.colab import files
from ktrain import text
from textblob_de.packages import pattern_de as patt
spacy.cli.download("de_core_news_sm")
nlp = spacy.load("de_core_news_sm")
from IPython.display import clear_output
clear_output()

In [0]:
#@title Daten und Modell initialisieren (dauert einen Moment) { display-mode: "form" }
#@markdown Load types and models (this may take while)
df = pd.read_parquet('df-types.parquet.gzip')
df2 = pd.DataFrame({'label':['sich','mich','dich','uns','sie'], 'type':['Präposition','Präposition','Präposition','Präposition','Präposition',]})
df = df.append(df2)
model = FastText.load_fasttext_format('cc.de.300.bin')
predictor = ktrain.load_predictor('predictor-crf')

In [0]:
#@title { display-mode: "form" }
def make_prediction(content):
  """
  Finds triplets in content-string.
    :param content: str, string to find triplets in
    :return predictions: array_like, list of triplets with content
      [{'OBJ':[],'REL':[],'VAL':[], 'CONTEXT':[content]}] 
  """
  try:
    prediction = predictor.predict(content)
  except ValueError:
    print('Value error for ' + str(content))
    return None
  except Exception as e:
    print("Something else went wrong: " + str(e))
    return None
  predicted_triplet = {'OBJ':[None],'REL':[None],'VAL':[None]}
  triplet_found = False
  index_range = list(range(len(prediction)))
  for i in index_range:
    if prediction[i][1].endswith('-obj'):
      objs = predicted_triplet['OBJ']
      tmp = prediction[i][0]
      look_for_triplets = True
      while look_for_triplets and i < index_range[-1]:
        i += 1
        if prediction[i][1] == 'I-obj':
          tmp = tmp + ' ' + prediction[i][0]
          index_range.remove(i)
        else:
          look_for_triplets = False
      objs.append(tmp)
      predicted_triplet['OBJ'] = objs
      triplet_found = True
    elif prediction[i][1].endswith('-val'):
      val = predicted_triplet['VAL']
      tmp = prediction[i][0]
      look_for_triplets = True
      while look_for_triplets and i < index_range[-1]:
        i += 1
        if prediction[i][1] == 'I-val':
          tmp = tmp + ' ' + prediction[i][0]
          index_range.remove(i)
        else:
          look_for_triplets = False
      val.append(tmp)
      predicted_triplet['VAL'] = val
      triplet_found = True
    elif prediction[i][1].endswith('-rel'):
      rel = predicted_triplet['REL']
      tmp = prediction[i][0]
      look_for_triplets = True
      while look_for_triplets and i < index_range[-1]:
        i += 1
        if prediction[i][1] == 'I-rel':
          tmp = tmp + ' ' + prediction[i][0]
          index_range.remove(i)
        else:
          look_for_triplets = False
      rel.append(tmp)
      predicted_triplet['REL'] = rel
      triplet_found = True
  if triplet_found:
    obj = predicted_triplet['OBJ'][1:] if len(predicted_triplet['OBJ']) > 1 else [None]
    rel = predicted_triplet['REL'][1:] if len(predicted_triplet['REL']) > 1 else [None]
    val = predicted_triplet['VAL'][1:] if len(predicted_triplet['VAL']) > 1 else [None]
    permutations = list(itertools.product(*[obj, rel, val]))
    predictions = [{'OBJ':[p[0]],'REL':[p[1]],'VAL':[p[2]], 'CONTEXT':[content]} for p in permutations]    
    return predictions
  else:
    return None

In [0]:
#@title { display-mode: "form" }
def get_baseline(content=None, url = None):
  """ Finds triplets, either in given string or in article on website (URL).
  :param content: str, optional, a text
  :param url: str, optional, an URL
  :return result: array_like, list of dictionary containing triplets
  """
  
  if url:
    url = url + "?action=render"
    response = requests.get(url)
    html = BeautifulSoup(response.text, 'html.parser')
    paragraphs = html.select("p")
    content = "".join([para.text for para in paragraphs])
  content = content.replace(";", ",")
  content = content.replace(" |", ",")
  content = re.sub(r"(\s+|\n)", " ", content)
  content = re.sub(r"[\'`\"']|(\[[^]]*\])|(,,)|(\( .*\?\/i\)\s)", "",content)
  content = content.replace('*', 'geboren am')
  content = content.replace('†', 'gestorben am')
  content = content.replace('•', '')
  nlp = spacy.load("de_core_news_sm")
  doc = nlp(content)
  sentences = list(doc.sents)
  result = []
  triplets = []
  for i in range(1, len(sentences) -1):
    search_text = sentences[i-1].text + ' ' + sentences[i].text + ' ' + sentences[i+1].text
    prediction = find_triplets(sentences[i-1:i+2])
    for pred in prediction:
      if pred['OBJ'] and pred['REL'] and pred['VAL'] and (pred['OBJ'], pred['REL'], pred['VAL']) not in triplets:
        print(pred)
        result.append(pred)
        triplets.append((pred['OBJ'], pred['REL'], pred['VAL']))
  return result

In [0]:
#@title { display-mode: "form" }
def make_prediction_with_url(content=None, url=None):
  """ Wrapper object to apply ktrain-predictor to text. Either in given string 
    or in article on website (URL).
  :param content: str, optional, a text
  :param url: str, optional, an URL to retrieve the article from
  :param predictor: ktrain-predictor object
  :return result: array_like, list of dictionary containing triplets
  """
  
  if url:
    url = url + "?action=render"
    response = requests.get(url)
    html = BeautifulSoup(response.text, 'html.parser')
    paragraphs = html.select("p")
    content = "".join([para.text for para in paragraphs])
  content = content.replace(";", ",")
  content = content.replace(" |", ",")
  content = re.sub(r"(\s+|\n)", " ", content)
  content = re.sub(r"[\'`\"']|(\[[^]]*\])|(,,)|(\( .*\?\/i\)\s)", "",content)
  content = content.replace('(', '( ')
  content = content.replace(')', ' )')
  content = content.replace('„', '')
  content = content.replace('”', '')
  content = content.replace('“', '')
  content = content.replace('‘', '')
  content = re.sub(r"\d-\d", "\d - \d", content)
  content = re.sub(r"\d–\d", "\d – \d", content)
  content = content.replace('•', '')
  nlp = spacy.load("de_core_news_sm")
  doc = nlp(content)
  sentences = list(doc.sents)
  result = []
  triplets = []
  for i in range(1, len(sentences) -1):
    search_text = sentences[i-1].text + ' ' + sentences[i].text + ' ' + sentences[i+1].text
    predictions = make_prediction(search_text)
    if predictions:
      for p in predictions:
        for s in [sentences[i-1], sentences[i], sentences[i+1]]:
          prediction = resolve_prediction(s, p)
          if prediction and (prediction['OBJ'], prediction['REL'], prediction['VAL']) not in triplets:
            result.append(prediction)
            triplets.append((prediction['OBJ'], prediction['REL'], prediction['VAL']))
            print(prediction)
  return result

In [0]:
#@title { display-mode: "form" }
def resolve_prediction(sentence, prediction):
  """ Helper method to fill incomplete triplets. 
  :param sentence: spacy-token string
  :param prediction: dict, an incomplete triplet
  :return prediction: dict, a complete triplet with object, relation, value
  """

  rel = [None]
  val = [None]
  obj = [None]
  # if nothing is present, do not make any prediction!
  if not prediction:
    return None
  # everything already present
  if prediction['OBJ']!=[None] and prediction['REL']!=[None] and prediction['VAL']!=[None]:
    return prediction
  #print(prediction['OBJ'], prediction['REL'], prediction['VAL'], sentence)
  sentence = nlp(sentence.text)
  # CASE 1: obj and rel are present
  if prediction['OBJ']!=[None] and prediction['REL']!=[None]:
    rel = prediction['REL'][0].split(' ')[0]
    val = [token.text for token in sentence if (token.dep_ == 'pd' and token.head.text == rel)] 
    if not val:
      val = [token.text for token in sentence if (token.dep_ == 'oc' and token.head.text == rel)]
    if not val:
      val = [token.text for token in sentence if (token.dep_ == 'oa' and token.head.text == rel)]
    if not val:
      val = [token.text for token in sentence if (token.dep_ == 'da' and token.head.text == rel)]
    if not val:
      val = [token.text for token in sentence if (token.dep_ == 'ag' and token.head.text == rel)]
    if not val:
      val = [token.text for token in sentence if (token.dep_ == 'nk' and token.head.text == rel)]
    if not val:
      return None
    prediction['VAL'] = [' '.join([v for v in val])]
  # CASE 2: obj and val are present
  elif prediction['OBJ']!=[None] and prediction['VAL']!=[None]:
    obj_item = prediction['OBJ'][0].split(' ')[-1]
    rel = [token.head.text for token in sentence if token.text == obj_item]
    if not rel:
      return None
    sent_head = get_head(rel[0], sentence)
    prediction['REL'] = [sent_head]
  # CASE 3: rel and val are present
  elif prediction['REL']!=[None] and prediction['VAL']!=[None]:
    rel = prediction['REL'][0].split(' ')[0]
    obj = [token.text for token in sentence if (token.dep_ == 'sb' and token.head.text == rel[0])]
    if not obj:
      return None
    prediction['OBJ'] = obj
  # CASE 4: only obj is present
  elif prediction['OBJ']!=[None]:
    obj_item = prediction['OBJ'][0].split(' ')[-1]
    rel = [token.head.text for token in sentence if token.text == obj_item]
    if rel:
      sent_head = get_head(rel[0], sentence)
      #[print(token, token.dep_, token.head.text) for token in sentence]
      val = [token.text for token in sentence if (token.dep_ == 'pd' and token.head.text == rel[0])] 
      if not val:
        val = [token.text for token in sentence if (token.dep_ == 'oc' and token.head.text == rel[0])]
      if not val:
        val = [token.text for token in sentence if (token.dep_ == 'oa' and token.head.text == rel[0])]
      if not val:
        val = [token.text for token in sentence if (token.dep_ == 'da' and token.head.text == rel[0])]
      if not val:
        val = [token.text for token in sentence if (token.dep_ == 'ag' and token.head.text == rel[0])]
      if not val:
        val = [token.text for token in sentence if (token.dep_ == 'nk' and token.head.text == rel[0])]
      if not val:
        return None
      prediction['REL'] = [sent_head]
      prediction['VAL'] = [' '.join([v for v in val])]
  # CASE 5: only rel is present
  elif prediction['REL']!=[None]:
    rel = prediction['REL'][0].split(' ')[0]
    obj = [token.text for token in sentence if (token.dep_ == 'sb' and token.head.text == rel)]
    if not obj:
      return None
    val = [token.text for token in sentence if (token.dep_ == 'pd' and token.head.text == rel)] 
    if not val:
      val = [token.text for token in sentence if (token.dep_ == 'oc' and token.head.text == rel)]
    if not val:
      val = [token.text for token in sentence if (token.dep_ == 'oa' and token.head.text == rel)]
    if not val:
      val = [token.text for token in sentence if (token.dep_ == 'da' and token.head.text == rel)]
    if not val:
      val = [token.text for token in sentence if (token.dep_ == 'ag' and token.head.text == rel)]
    if not val:
      val = [token.text for token in sentence if (token.dep_ == 'nk' and token.head.text == rel)]
    if not val:
      return None
    prediction['VAL'] = [' '.join([v for v in val])]
    prediction['OBJ'] = obj
  # CASE 6: only val is present
  elif prediction['VAL']!=[None]:
    for nc in sentence.noun_chunks:
      sent_head = get_head(nc.root.head.text, sentence)
      prediction['REL'] = [sent_head]
      if nc.root.dep_ == 'sb':
        prediction['OBJ'] = [nc.text]
        break
  if not prediction['OBJ'] or not prediction['REL'] or not prediction['VAL'] or prediction['OBJ'] == [None] or prediction['REL'] == [None] or prediction['VAL']==[None]:
    return None
  else:
    return prediction

In [0]:
#@title  { display-mode: "form" }
def get_head(head_obj, sentence):
  """ Helper method to retrieve the full head of a given object.
  :param head_obj: a head candidate
  :param sentence: spacy-token string
  :return final_head: str, the full head
  """
  final_head = head_obj
  for token in sentence:
    if token.text != head_obj and token.head.text == head_obj and token.tag_ in ['PTKVZ', 'ADJD']:
      final_head = final_head + ' ' + token.text.lower()
  return final_head

def find_triplets(sentences):
  """ Helper method to find triplets in given sentences. 
  :param sentences: array_like, list of spacy-sentences 
  :return values: array_like, list of found triplets in input-sentences
  """
  predictions = {}
  context = ' '.join([sent.text for sent in sentences])
  for sent in sentences:
    sent = nlp(sent.text)
    for nc in sent.noun_chunks:
      sent_head = get_head(nc.root.head.text, sent)
      if sent_head not in predictions:
        predictions[sent_head] = {'OBJ': None, 'REL': [sent_head], 'VAL': None, 'CONTEXT': [context]}
      if nc.root.dep_ == 'sb':
        tmp = predictions[sent_head]
        tmp['OBJ'] = [nc.text]
        predictions[sent_head] = tmp
      elif not predictions[sent_head]['VAL'] and nc.root.dep_ == 'pd': 
        tmp = predictions[sent_head]
        tmp['VAL'] = [nc.text]
        predictions[sent_head] = tmp
      elif not predictions[sent_head]['VAL'] and nc.root.dep_ == 'oa': 
        tmp = predictions[sent_head]
        tmp['VAL'] = [nc.text]
        predictions[sent_head] = tmp
      elif not predictions[sent_head]['VAL'] and nc.root.dep_ == 'da': 
        tmp = predictions[sent_head]
        tmp['VAL'] = [nc.text]
        predictions[sent_head] = tmp
      elif not predictions[sent_head]['VAL'] and nc.root.dep_ == 'ag': 
        tmp = predictions[sent_head]
        tmp['VAL'] = [nc.text]
        predictions[sent_head] = tmp
      elif not predictions[sent_head]['VAL'] and nc.root.dep_ == 'nk':
        tmp = predictions[sent_head]
        tmp['VAL'] = [nc.text]
        predictions[sent_head] = tmp
  values = list(predictions.values())
  return values

In [0]:
#@title --
#@markdown ## ![Re-Start](https://drive.google.com/uc?id=111kJPmPWeqSrNxQkE252dQNShz0pM7VG)Bei jeder weiteren Ausführung: Zelle anwählen und CTRL+F10. { display-mode: "form" }
#@markdown Every other execution: Select this cell. Then hit CTRL+F10.

In [0]:
#@title Triplets extrahieren (Baseline und mit Modell) { display-mode: "form" }
#@markdown Extract triplets (baseline and model)
if URL:
  baseline_predictions = get_baseline(url=URL)
  predictions = make_prediction_with_url(url=URL)
  clear_output()
elif content and len(content) > 1:
  baseline_predictions = get_baseline(content)
  predictions = make_prediction_with_url(content)
  clear_output()
else:
  print("Bitte spezifieren Sie eine URL oder laden Sie einen Text hoch!\n")
  print("Please specify an URL or upload a text!")

In [0]:
#@title { display-mode: "form" }
def replace(predictions, replacement, object_type='neutral'):
  """ Method that replaces 
  :param predictions: dict, triplets
  :param replacement: str
  :param object_type: str, indicates gender of object, default='neutral'
  """
  for pred in predictions:
    tmp = pred['OBJ']
    if object_type == 'male':
      splits = replacement.split(' ')
      to_replace = [splits[-1], 'er', 'Er']
      tmp = [replacement if x in to_replace else x for x in tmp]
      to_replace = ['sein', 'seine', 'Sein', 'Seine']
      tmp = [replacement + 's' if x in to_replace else x for x in tmp]
    elif object_type == 'female':
      splits = replacement.split(' ')
      to_replace = [splits[-1], 'sie', 'Sie', 'ihr']
      tmp = [replacement if x in to_replace else x for x in tmp]
      to_replace = ['ihr', 'ihre', 'Ihr', 'Ihre']
      tmp = [replacement + 's' if x in to_replace else x for x in tmp]
    else:
      to_replace = ['es', 'Es', 'er', 'Er']
      tmp = [replacement if x in to_replace else x for x in tmp]
    pred['OBJ'] = tmp
    rel = pred['REL']
    rel = ['geboren' if x == '*' else x for x in rel]
    rel = ['gestorben' if x == '†' else x for x in rel]
    pred['REL'] = rel
  return predictions

def filter_triplets(predictions):
  """ Filter out the bad triplets according to their distance to the median.
  :param predictions: array_like, list of triplets-dict
  :return result: array_like, filtered triplets-dict
  """
  max_distance = 0.0035
  median=np.array([0.00080595, 0.00218227, 0.00070459])
  result = []
  for pred in predictions:
    try:
      x = np.mean(model[pred['OBJ'][0].replace(' ','')])
      y = np.mean(model[pred['REL'][0].replace(' ','')])
      z = np.mean(model[pred['VAL'][0].replace(' ','')])
      vec = np.array([x, y, z])
      distance = np.linalg.norm(median - vec)
      if distance <= max_distance:
        result.append(pred)
    except (Exception) as e:
      print(str(e))
      continue
  return result

In [0]:
#@title Ersetzen von koreferenten Objekten (falls gewünscht) und entfernen von "schlechten" Triplets { display-mode: "form" }
#@markdown If desired, replace co-referent objects and remove "bad" triplets
if 'replace_objects' in globals() and replace_objects:
  baseline_predictions = replace(baseline_predictions, replacement, object_type = gender_dropdown)
  predictions = replace(predictions, replacement, object_type = gender_dropdown)
baseline_predictions = filter_triplets(baseline_predictions)
predictions = filter_triplets(predictions)

In [0]:
#@title { display-mode: "form" }
class CustomParserInfo(parserinfo):
    MONTHS = [("Januar"), ("Februar", "Feb"), ("März", "Mär"), ("April", "Apr"), ("Mai", "Mai"), 
              ("Juni", "Jun"), ("Juli", "Jul"), ("August", "Aug"), ("September", "Sep"), 
              ("Oktober", "Okt"), ("November", "Nov"), ("Dezember", "Dez")]

def is_date(label, fuzzy=False):
    """
    Return whether the string can be interpreted as a date.
    :param label: str, string to check for date
    :param fuzzy: bool, ignore unknown tokens in string if True
    :return: bool, whether string is a date
    """
    try: 
        parse(label, fuzzy=fuzzy, parserinfo=CustomParserInfo())
        return True

    except ValueError:
        return False

def lookup_category(label):
  """Method to categorize the given label. 

  :param label: str, the string to categorize
  :return: array_like;
    ['Zeitpunkt'] if date
    ['Zahl'] if digit
    [list of types] if label exists in types-table (types data frame) 
    [list of types] if one of the 20 most similar labels exists in types-table 
    ['Ding'] if nothing was found  """
    
  is_digit = re.match('(^\d*$|^\d*\.\d*$)', label)
  if not is_digit and is_date(label):
    return ['Zeitpunkt']
  elif is_digit:
    return ['Zahl']
  if label in df['label'].values:
    types = df.loc[df['label'] == label, 'type'].values
    return list(set(types))
  else:
    try:
      lookup = model.wv.most_similar(positive=[label], topn=20)
      for x in lookup:
        if x[0] in df['label'].values:
          types = df.loc[df['label'] == x[0], 'type'].values
          return list(set(types))
      return ['Ding']
    except:
      return ['Ding']

In [0]:
#@title { display-mode: "form" }
auxiliary_dict = {'geboren':'wurde ','gestorben':'ist ','geschrieben':'wurde/hat ','gegründet':'wurde/hat ', 'getauft':'wurde/hat '}
gap = '_____________'
mc = ['A: ', 'B: ', 'C: ', 'D: ']

def upper_first(s):
    """
    Return a copy of the string s with its first character
    capitalized.
    :param s: str
    :return str:
    """
    return s[0].upper() + s[1:]

def get_equivalents(pred, obj_cat, val_cat):
  """Retrieves the three most similar objects and values based on its type(s).

  :param pred: dict, predicted triplet
  :param obj_cat: array_like, list of object's types
  :param val_cat: array_like, list of value's types
  :return: tuple, tuple containing object-replacements and value-replacements"""
  
  if np.where(obj_cat == 'Mensch') and len(obj_cat)>1:
    np.delete(obj_cat, np.where(obj_cat == 'Mensch'))
  if np.where(val_cat == 'Mensch') and len(val_cat)>1:
    np.delete(val_cat, np.where(val_cat == 'Mensch'))
  if 'Mensch' in obj_cat:
    obj_eq = list(set(df.loc[(df['type'] == 'Mensch') & (df['label'] != pred['OBJ'][0]), 'label'].values))
    obj_eq = random.sample(obj_eq, 3)
  elif 'Zeitpunkt' in obj_cat:
    prefix = 'am ' if 'am ' in pred['OBJ'][0] else ''
    base = dateparser.parse(pred['OBJ'][0].replace('am ', ''), date_formats=['%d.%m.%Y'])
    pred['OBJ'] = [prefix + base.strftime("%d.%m.%Y")]
    date_list = [base + datetime.timedelta(days=x) for x in [random.randrange(-100000, 100000, 1), random.randrange(-10000, 10000, 1), random.randrange(-100, 100, 1)]]
    obj_eq = [prefix + date.strftime("%d.%m.%Y") for date in date_list]
  elif 'Zahl' in obj_cat:
    base = float(pred['OBJ'][0]) if '.' in pred['OBJ'][0] else int(pred['OBJ'][0])
    obj_eq = [str(base + x) for x in [random.randrange(-1000, 1000, 1), random.randrange(-100, 100, 1), random.randrange(-10, 10, 1)]]
  else:
    obj_equivalents = [df.loc[df['type'] == oc, 'label'].values for oc in obj_cat]
    obj_equivalents = [item for sublist in obj_equivalents for item in sublist]
    obj_freq = {} 
    for item in obj_equivalents: 
      if item != pred['OBJ'][0]:
        obj_freq[item] = obj_equivalents.count(item) 
    obj_eq = sorted(obj_freq, key=obj_freq.get, reverse = True)
  if 'Mensch' in val_cat:
    val_eq = list(set(df.loc[(df['type'] == 'Mensch') & (df['label'] != pred['VAL'][0]), 'label'].values))
    val_eq = random.sample(obj_eq, 3)
  elif 'Zeitpunkt' in val_cat:
    prefix = 'am ' if 'am ' in pred['VAL'][0] else ''
    base = dateparser.parse(pred['VAL'][0].replace('am ', ''), date_formats=['%d.%m.%Y'])
    pred['VAL'] = [prefix + base.strftime("%d.%m.%Y")]
    date_list = [base + datetime.timedelta(days=x) for x in [random.randrange(-100000, 100000, 1), random.randrange(-10000, 10000, 1), random.randrange(-100, 100, 1)]]
    val_eq = [prefix + date.strftime("%d.%m.%Y")for date in date_list]
  elif 'Zahl' in val_cat:
    base = float(pred['VAL'][0]) if '.' in pred['VAL'][0] else int(pred['VAL'][0])
    val_eq = [str(base + x) for x in [random.randrange(-1000, 1000, 1), random.randrange(-100, 100, 1), random.randrange(-10, 10, 1)]]
  else:
    val_equivalents = [df.loc[df['type'] == vc, 'label'].values for vc in val_cat]
    val_equivalents = [item for sublist in val_equivalents for item in sublist]
    val_freq = {}
    for item in val_equivalents: 
      if item != pred['VAL'][0]:
        val_freq[item] = val_equivalents.count(item)
    val_eq = sorted(val_freq, key=val_freq.get, reverse = True)
  return (obj_eq[:3], val_eq[:3])

def apply_rule1_extended(pred, rel, obj_cat, val_cat):
  """
  #1 <Object> hat ein(e/en) <Relation (NN/NE)> von <Value (Number)>.
    --> Mount Everest hat eine Höhe von 8848.

  :param pred: dict, predicted triplet
  :param rel: spacy token
  :param obj_cat: array_like, list of object's types
  :param val_cat: array_like, list of value's types
  :return true_stmt: str, statement which is true
  :return false_stmt1: str, statement with a wrong object
  :return false_stmt2: str, statement with a wrong value
  :return gap_obj: str, statement with object blanked out
  :return gap_val: str, statement with value blanked out
  :return mc_obj: array_like, list of possible object-replacements
  :return mc_val: array_like, list of possible value-replacements
  """
  equivalents = get_equivalents(pred, obj_cat, val_cat)
  obj_description = ' (' + '/'.join([word for word in obj_cat]) + ') ' if 'Ding' not in obj_cat else ''
  val_description = ' (' + '/'.join([word for word in val_cat]) + ') ' if 'Ding' not in obj_cat else ''
  noun = [token.text for token in rel if token.tag_ == 'NN' or token.tag_ == 'NE'][0]
  gender = lookup_gender(noun)
  suffixes = {'m':'en ', 'f':'e ', 'n':' ', 'UNK':'/e(n) '}

  true_stmt = upper_first(pred['OBJ'][0]) + ' hat ein' + suffixes[gender] + rel.text + ' von ' + pred['VAL'][0] + '.'
  false_stmt1 = upper_first(equivalents[0][0]) + ' hat ein' + suffixes[gender] + rel.text + ' von ' + pred['VAL'][0] + '.' if equivalents[0] else None
  false_stmt2 = upper_first(pred['OBJ'][0]) + ' hat ein' + suffixes[gender] + rel.text + ' von ' + equivalents[1][0] + '.' if equivalents[1] else None
  gap_obj = gap + obj_description + ' hat ein' + suffixes[gender] + rel.text + ' von ' + pred['VAL'][0] + '.'
  gap_val = upper_first(pred['OBJ'][0]) + ' hat ein' + suffixes[gender] + rel.text + ' von ' + gap + val_description + '.'
  mc_obj = equivalents[0] + pred['OBJ'] if equivalents[0] else []
  mc_val = equivalents[1] + pred['VAL'] if equivalents[1] else []
  random.shuffle(mc_obj)
  random.shuffle(mc_val)
  mc_obj = ' '.join([mc[i] + mc_obj[i] for i in range(len(mc_obj))])
  mc_val = ' '.join([mc[i] + mc_val[i] for i in range(len(mc_val))])
  return (true_stmt, false_stmt1, false_stmt2, gap_obj, gap_val, mc_obj, mc_val)

def apply_rule2_extended(pred, rel, obj_cat, val_cat, is_template_a):
  """
  #2a <Object> ist <Relation (NN/NE)> <Relation (APPR/APPRART/PTKVZ)> <Value>.
	  --> Christopher Douglas Adams ist (der) Vater von Douglas Adams.
  #2b <Object> hat als <Relation (NN/NE)> <Value>.
	  --> "Eine kleine Nachtmusik" hat als Komponist Wolfgang Amadeus Mozart.

  :param pred: dict, predicted triplet
  :param rel: spacy token
  :param obj_cat: array_like, list of object's types
  :param val_cat: array_like, list of value's types
  :param is_template_a: bool
  :return true_stmt: str, statement which is true
  :return false_stmt1: str, statement with a wrong object
  :return false_stmt2: str, statement with a wrong value
  :return gap_obj: str, statement with object blanked out
  :return gap_val: str, statement with value blanked out
  :return mc_obj: array_like, list of possible object-replacements
  :return mc_val: array_like, list of possible value-replacements
  """
  equivalents = get_equivalents(pred, obj_cat, val_cat)
  obj_description = ' (' + '/'.join([word for word in obj_cat[1:]]) + ') ' if len(obj_cat) > 1 else ''
  val_description = ' (' + '/'.join([word for word in val_cat[1:]]) + ') ' if len(val_cat) > 1 else ''
  if is_template_a:
    rel_head = ' '.join([token.text for token in rel if token.tag_ == 'NN' or token.tag_ == 'NE'])
    rel_head = rel_head + ' ' if len(rel_head) > 0 else ''
    rel_tail = ' '.join([token.text for token in rel if token.tag_ == 'PTKVZ' or token.tag_=='APPR' or token.tag_=='APPRART'])
    
    true_stmt = upper_first(pred['OBJ'][0]) + ' ist ' + rel_head + rel_tail + ' ' + pred['VAL'][0] + '.'
    false_stmt1 = upper_first(equivalents[0][0]) + ' ist ' + rel_head + rel_tail + ' ' + pred['VAL'][0] + '.' if equivalents[0] else None
    false_stmt2 = upper_first(pred['OBJ'][0]) + ' ist ' + rel_head + rel_tail + ' ' + equivalents[1][0] + '.' if equivalents[1] else None
    gap_obj = gap + obj_description + ' ist ' + rel_head + rel_tail + ' ' + pred['VAL'][0] + '.'
    gap_val = upper_first(pred['OBJ'][0]) + ' ist ' + rel_head + rel_tail + ' ' + gap + val_description + '.'
  else:
    true_stmt = upper_first(pred['OBJ'][0]) + ' hat als ' + rel.text + ' ' + pred['VAL'][0] + '.'
    false_stmt1 = upper_first(equivalents[0][0]) + ' hat als ' + rel.text + ' ' + pred['VAL'][0] + '.' if equivalents[0] else None
    false_stmt2 = upper_first(pred['OBJ'][0]) + ' hat als ' + rel.text + ' ' + equivalents[1][0] + '.' if equivalents[1] else None
    gap_obj = gap + obj_description + ' hat als ' + rel.text + ' ' + pred['VAL'][0] + '.'
    gap_val = upper_first(pred['OBJ'][0]) + ' hat als ' + rel.text + ' ' + gap + val_description + '.'
  mc_obj = equivalents[0] + pred['OBJ'] if equivalents[0] else []
  mc_val = equivalents[1] + pred['VAL'] if equivalents[1] else []
  random.shuffle(mc_obj)
  random.shuffle(mc_val)
  mc_obj = ' '.join([mc[i] + mc_obj[i] for i in range(len(mc_obj))])
  mc_val = ' '.join([mc[i] + mc_val[i] for i in range(len(mc_val))])
  return (true_stmt, false_stmt1, false_stmt2, gap_obj, gap_val, mc_obj, mc_val)

def apply_rule3_extended(pred, rel, obj_cat, val_cat):
  """
   #3 <Object> wird/ist/wurde/hat <Relation (!VVPP)> <Value> <Relation (VVPP)>.
	  --> Am 11.März 1952 wurde Douglas Adams geboren.
	  --> Kurt Cobain ist am 5. April 1994 gestorben.
	  --> Kopfschmerzen wird mit Ibuprofen behandelt.

  :param pred: dict, predicted triplet
  :param rel: spacy token
  :param obj_cat: array_like, list of object's types
  :param val_cat: array_like, list of value's types
  :return true_stmt: str, statement which is true
  :return false_stmt1: str, statement with a wrong object
  :return false_stmt2: str, statement with a wrong value
  :return gap_obj: str, statement with object blanked out
  :return gap_val: str, statement with value blanked out
  :return mc_obj: array_like, list of possible object-replacements
  :return mc_val: array_like, list of possible value-replacements
  """
  equivalents = get_equivalents(pred, obj_cat, val_cat)
  obj_description = ' (' + '/'.join([word for word in obj_cat[1:]]) + ') ' if len(obj_cat) > 1 else ''
  val_description = ' (' + '/'.join([word for word in val_cat[1:]]) + ') ' if len(val_cat) > 1 else ''
  rel_head = ' '.join([token.text for token in rel if token.tag_ != 'VVPP'])
  rel_tail = ' '.join([token.text for token in rel if token.tag_ == 'VVPP'])
  rel_head = rel_head + ' ' if len(rel_head) > 0 else ''
  auxiliary = auxiliary_dict[rel_tail] if rel_tail in auxiliary_dict else 'wird/wurde/hat/ist '

  true_stmt = upper_first(pred['OBJ'][0]) + ' ' + auxiliary + rel_head + pred['VAL'][0] + ' ' + rel_tail + '.'
  false_stmt1 = upper_first(equivalents[0][0]) +  ' ' + auxiliary + rel_head + pred['VAL'][0] + ' ' + rel_tail + '.' if equivalents[0] else None
  false_stmt2 = upper_first(pred['OBJ'][0]) +  ' ' + auxiliary + rel_head + equivalents[1][0] + ' ' + rel_tail + '.' if equivalents[1] else None
  gap_obj = gap + obj_description + ' ' + auxiliary + rel_head + pred['VAL'][0] + ' ' + rel_tail + '.'
  gap_val = upper_first(pred['OBJ'][0]) + ' ' + auxiliary + rel_head + gap + val_description + ' ' + rel_tail + '.'
  mc_obj = equivalents[0] + pred['OBJ'] if equivalents[0] else []
  mc_val = equivalents[1] + pred['VAL'] if equivalents[1] else []
  random.shuffle(mc_obj)
  random.shuffle(mc_val)
  mc_obj = ' '.join([mc[i] + mc_obj[i] for i in range(len(mc_obj))])
  mc_val = ' '.join([mc[i] + mc_val[i] for i in range(len(mc_val))])
  return (true_stmt, false_stmt1, false_stmt2, gap_obj, gap_val, mc_obj, mc_val)

def apply_rule4_extended(pred, rel, obj_cat, val_cat, is_template_a):
  """
  #4a <Object> <Relation V*FIN> <Value> <Relation (PTKVZ)>.
	  --> "Harry Potter und der Stein der Weisen" schrieb Joanne K. Rowling nieder.
	
	#4b <Object> <Relation V*FIN> <Value>?  
    --> "Harry Potter und der Stein der Weisen" schrieb Joanne K. Rowling.

  :param pred: dict, predicted triplet
  :param rel: spacy token
  :param obj_cat: array_like, list of object's types
  :param val_cat: array_like, list of value's types
  :param is_template_a: bool
  :return true_stmt: str, statement which is true
  :return false_stmt1: str, statement with a wrong object
  :return false_stmt2: str, statement with a wrong value
  :return gap_obj: str, statement with object blanked out
  :return gap_val: str, statement with value blanked out
  :return mc_obj: array_like, list of possible object-replacements
  :return mc_val: array_like, list of possible value-replacements
  """  
  equivalents = get_equivalents(pred, obj_cat, val_cat)
  obj_description = ' (' + '/'.join([word for word in obj_cat[1:]]) + ') ' if len(obj_cat) > 1 else ''
  val_description = ' (' + '/'.join([word for word in val_cat[1:]]) + ') ' if len(val_cat) > 1 else ''
  if is_template_a:
    rel_head = ' '.join([token.text for token in rel if token.tag_ != 'PTKVZ'])
    rel_tail = ' '.join([token.text for token in rel if token.tag_ == 'PTKVZ'])

    true_stmt = upper_first(pred['OBJ'][0]) + ' ' + rel_head + ' ' + pred['VAL'][0] + ' ' + rel_tail + '.'
    false_stmt1 = upper_first(equivalents[0][0]) + ' ' + rel_head + ' ' + pred['VAL'][0] + ' ' + rel_tail + '.' if equivalents[0] else None
    false_stmt2 = upper_first(pred['OBJ'][0]) + ' ' + rel_head + ' ' + equivalents[1][0] + ' ' + rel_tail + '.' if equivalents[1] else None
    gap_obj = gap + obj_description + ' ' + rel_head + ' ' + pred['VAL'][0] + ' ' + rel_tail + '.'
    gap_val = upper_first(pred['OBJ'][0]) + ' ' + rel_head + ' ' + gap + val_description + ' ' + rel_tail + '.'
  else:
    true_stmt = upper_first(pred['OBJ'][0]) + ' ' + rel.text + ' ' + pred['VAL'][0] + '.'
    false_stmt1 = upper_first(equivalents[0][0]) + ' ' + rel.text + ' ' + pred['VAL'][0] + '.' if equivalents[0] else None
    false_stmt2 = upper_first(pred['OBJ'][0]) + ' ' + rel.text + ' ' + equivalents[1][0] + '.' if equivalents[1] else None
    gap_obj = gap + obj_description + ' ' + rel.text + ' ' + pred['VAL'][0] + '.'
    gap_val = upper_first(pred['OBJ'][0]) + ' ' + rel.text + ' ' + gap + val_description + '.'
  mc_obj = equivalents[0] + pred['OBJ'] if equivalents[0] else []
  mc_val = equivalents[1] + pred['VAL'] if equivalents[1] else []
  random.shuffle(mc_obj)
  random.shuffle(mc_val)
  mc_obj = ' '.join([mc[i] + mc_obj[i] for i in range(len(mc_obj))])
  mc_val = ' '.join([mc[i] + mc_val[i] for i in range(len(mc_val))])
  return (true_stmt, false_stmt1, false_stmt2, gap_obj, gap_val, mc_obj, mc_val)

def apply_rule5_extended(pred, rel, obj_cat, val_cat, is_template_a):
  """
  Clean up-rule. Verb is most probably predicted falsely!
  #5a <Object> <Relation (!PTKVZ)> <Value> <Relation (PTKVZ)>.
	#5b <Object> <Relation> <Value>.

  :param pred: dict, predicted triplet
  :param rel: spacy token
  :param obj_cat: array_like, list of object's types
  :param val_cat: array_like, list of value's types
  :param is_template_a: bool
  :return true_stmt: str, statement which is true
  :return false_stmt1: str, statement with a wrong object
  :return false_stmt2: str, statement with a wrong value
  :return gap_obj: str, statement with object blanked out
  :return gap_val: str, statement with value blanked out
  :return mc_obj: array_like, list of possible object-replacements
  :return mc_val: array_like, list of possible value-replacements
  """
  equivalents = get_equivalents(pred, obj_cat, val_cat)
  obj_description = ' (' + '/'.join([word for word in obj_cat[1:]]) + ') ' if len(obj_cat) > 1 else ''
  val_description = ' (' + '/'.join([word for word in val_cat[1:]]) + ') ' if len(val_cat) > 1 else ''
  if is_template_a:
    rel_head = ' '.join([token.text for token in rel if token.tag_ != 'PTKVZ'])
    rel_tail = ' '.join([token.text for token in rel if token.tag_ == 'PTKVZ'])

    true_stmt = upper_first(pred['OBJ'][0]) + ' ' + rel_head + ' ' + pred['VAL'][0] + ' ' + rel_tail + '.'
    false_stmt1 = upper_first(equivalents[0][0]) + ' ' + rel_head + ' ' + pred['VAL'][0] + ' ' + rel_tail + '.' if equivalents[0] else None
    false_stmt2 = upper_first(pred['OBJ'][0]) + ' ' + rel_head + ' ' + equivalents[1][0] + ' ' + rel_tail + '.' if equivalents[1] else None
    gap_obj = gap + obj_description + ' ' + rel_head + ' ' + pred['VAL'][0] + ' ' + rel_tail + '.'
    gap_val = upper_first(pred['OBJ'][0]) + ' ' + rel_head + ' ' + gap + val_description + ' ' + rel_tail + '.'
  else:
    true_stmt = upper_first(pred['OBJ'][0]) + ' ' + rel.text + ' ' + pred['VAL'][0] + '.'
    false_stmt1 = upper_first(equivalents[0][0]) + ' ' + rel.text + ' ' + pred['VAL'][0] + '.' if equivalents[0] else None
    false_stmt2 = upper_first(pred['OBJ'][0]) + ' ' + rel.text + ' ' + equivalents[1][0] + '.' if equivalents[1] else None
    gap_obj = gap + obj_description + ' ' + rel.text + ' ' + pred['VAL'][0] + '.'
    gap_val = upper_first(pred['OBJ'][0]) + ' ' + rel.text + ' ' + gap + val_description + '.'
  mc_obj = equivalents[0] + pred['OBJ'] if equivalents[0] else []
  mc_val = equivalents[1] + pred['VAL'] if equivalents[1] else []
  random.shuffle(mc_obj)
  random.shuffle(mc_val)
  mc_obj = ' '.join([mc[i] + mc_obj[i] for i in range(len(mc_obj))])
  mc_val = ' '.join([mc[i] + mc_val[i] for i in range(len(mc_val))])
  return (true_stmt, false_stmt1, false_stmt2, gap_obj, gap_val, mc_obj, mc_val)

In [0]:
#@title { display-mode: "form" }
auxiliary_dict = {'geboren':'wurde ','gestorben':'ist ','geschrieben':'wurde/hat ','gegründet':'wurde/hat ', 'getauft':'wurde/hat '}
locations = ['Metropole', 'Bundesland','County', 'Land','Kanton','Dorf','Gemeinde']

def lookup_gender(input):
  """Helper method to lookup gender of given input-word(s). 
  :param input: str
  :return: str, 
    m if masculin
    f if feminine
    n if neutrum
    UNK if unknown"""
    
  word = input.replace('ö', 'oe')
  word = word.replace('ä', 'ae')
  word = word.replace('ü', 'ue')
  try:
    w = duden.get(word.split(' ')[-1])
    suffixes = {'der':'m', 'die':'f', 'das':'n'}
    if w and w.article:
      return suffixes[w.article]
    else:
      return 'UNK'
  except:
    return patt.gender(input) if patt.gender(input) else 'UNK'

def get_question_word(obj_cat, val_cat):
  """Generate question word based on category and gender of given object.
  :param obj_cat: array_like, list of object's types
  :param val_cat: array_like, list of value's types
  :return: str, question word and optional description of object"""

  if 'Mensch' in obj_cat:
    question_word = 'Wer '
  elif 'Familienname' in obj_cat or 'männlicher Vorname' in obj_cat or 'weiblicher Vorname' in obj_cat:
    question_word = 'Wer '
    obj_cat = np.insert(obj_cat, 0, 'Mensch')
  elif ('Mensch' in val_cat and obj_cat[0].endswith('tadt')) or ('Mensch' in val_cat and obj_cat[0].endswith('taat')) or ('Mensch' in val_cat and obj_cat[0] in locations):
    question_word = 'Wo '
    obj_cat = np.insert(obj_cat, 0, 'Ort')
  elif 'Zeitpunkt' in obj_cat or 'Zeitpunkt innerhalb eines wiederkehrenden Zeitrahmens' in obj_cat:
    question_word = 'Wann '
  elif 'Ding' in obj_cat:
    question_word = 'Wer oder was ' 
  else:
    gender = lookup_gender(obj_cat[0])
    suffixes = {'m':'r ', 'f':' ', 'n':'s ', 'UNK':'(-/r/s) '}
    question_word = 'Welche' + suffixes[gender] + obj_cat[0] + ' '
  description = '(' + '/'.join([word for word in obj_cat[1:]]) + ') ' if len(obj_cat) > 1 else ''
  return question_word + description

def apply_rule1(pred, rel, obj_cat, val_cat):
  """
  #1 Welche(r/s) <Object-Type> hat ein(e/en) <Relation (NN/NE)> von <Value (Number)>? 
	  --> Welcher Berg hat eine Höhe von 8848?

  :param pred: dict, predicted triplet
  :param rel: spacy token
  :param obj_cat: array_like, list of object's types
  :param val_cat: array_like, list of value's types
  :return: str, generated question
  """
  noun = [token.text for token in rel if token.tag_ == 'NN' or token.tag_ == 'NE'][0]
  question_word = get_question_word(obj_cat, val_cat)
  gender = lookup_gender(noun)
  suffixes = {'m':'en ', 'f':'e ', 'n':' ', 'UNK':'/e(n) '}
  question = question_word + 'hat ein' + suffixes[gender] + rel.text + ' von ' + pred['VAL'][0] + '?'
  return question

def apply_rule2(pred, rel, obj_cat, val_cat, is_template_a):
  """
  #2a Welche(r/s) <Object-Type> ist <Relation (NN/NE)> <Relation (APPR/APPRART/PTKVZ)> <Value>?
	  --> Welcher Mensch ist (der) Vater von Douglas Adams?
	  --> Wer ist (der) Vater von Douglas Adams?
  #2b Welche(r/s) <Object-Type> hat als <Relation (NN/NE)> <Value>? 
	  --> Welches Werk hat als Komponist Wolfgang Amadeus Mozart?

  :param pred: dict, predicted triplet
  :param rel: spacy token
  :param obj_cat: array_like, list of object's types
  :param val_cat: array_like, list of value's types
  :param is_template_a: bool
  :return: str, generated question
  """
  question_word = get_question_word(obj_cat, val_cat)
  if is_template_a:
    rel_head = ' '.join([token.text for token in rel if token.tag_ == 'NN' or token.tag_ == 'NE'])
    rel_head = rel_head + ' ' if len(rel_head) > 0 else ''
    rel_tail = ' '.join([token.text for token in rel if token.tag_ == 'PTKVZ' or token.tag_=='APPR' or token.tag_=='APPRART'])
    return question_word + 'ist ' + rel_head + rel_tail + ' ' + pred['VAL'][0] + '?'
  else:
    return question_word + 'hat als ' + rel.text + ' ' + pred['VAL'][0] + '?'

def apply_rule3(pred, rel, obj_cat, val_cat):
  """
   #3 Welche(r/s) <Object-Type> wird/ist/wurde/hat <Relation (!VVPP)> <Value> <Relation (VVPP)>?
	  --> Welcher Zeitpunkt wurde Douglas Adams geboren?
	  --> Wann wurde Douglas Adams geboren?
	  --> Wer ist am 5. April 1994 gestorben?
	  --> Welche Krankheit wird mit Ibuprofen behandelt?

  :param pred: dict, predicted triplet
  :param rel: spacy token
  :param obj_cat: array_like, list of object's types
  :param val_cat: array_like, list of value's types
  :return: str, generated question
  """
  question_word = get_question_word(obj_cat, val_cat)
  rel_head = ' '.join([token.text for token in rel if token.tag_ != 'VVPP'])
  rel_tail = ' '.join([token.text for token in rel if token.tag_ == 'VVPP'])
  rel_head = rel_head + ' ' if len(rel_head) > 0 else ''
  auxiliary = auxiliary_dict[rel_tail] if rel_tail in auxiliary_dict else 'wird/wurde/hat/ist '
  return question_word + auxiliary + rel_head + pred['VAL'][0] + ' ' + rel_tail + '?'

def apply_rule4(pred, rel, obj_cat, val_cat, is_template_a):
  """
  #4a Welche(r/s) <Object-Type> <Relation V*FIN> <Value> <Relation (PTKVZ)>? 
	  --> Welches Buch schrieb Joanne K. Rowling nieder?
	
	#4b Welche(r/s) <Object-Type> <Relation V*FIN> <Value>?  
    --> Welches Buch schrieb Joanne K. Rowling?

  :param pred: dict, predicted triplet
  :param rel: spacy token
  :param obj_cat: array_like, list of object's types
  :param val_cat: array_like, list of value's types
  :param is_template_a: bool
  :return: str, generated question
  """
  question_word = get_question_word(obj_cat, val_cat)
  if is_template_a:
    rel_head = ' '.join([token.text for token in rel if token.tag_ != 'PTKVZ'])
    rel_tail = ' '.join([token.text for token in rel if token.tag_ == 'PTKVZ'])
    return question_word + rel_head + ' ' + pred['VAL'][0] + ' ' + rel_tail + '?'
  else:
    return question_word + rel.text + ' ' + pred['VAL'][0] + '?'


def apply_rule5(pred, rel, obj_cat, val_cat, is_template_a):
  """
  Clean up-rule. Verb is most probably predicted falsely!
  #5a Welche(r/s) <Object-Type> <Relation (!PTKVZ)> <Value> <Relation (PTKVZ)>? 
	#5b Welche(r/s) <Object-Type> <Relation> <Value>? 

  :param pred: dict, predicted triplet
  :param rel: spacy token
  :param obj_cat: array_like, list of object's types
  :param val_cat: array_like, list of value's types
  :param is_template_a: bool
  :return: str, generated question
  """
  question_word = get_question_word(obj_cat, val_cat)
  if is_template_a:
    rel_head = ' '.join([token.text for token in rel if token.tag_ != 'PTKVZ'])
    rel_tail = ' '.join([token.text for token in rel if token.tag_ == 'PTKVZ'])
    return question_word + rel_head + ' ' + pred['VAL'][0] + ' ' + rel_tail + '?'
  else:
    return question_word + rel.text + ' ' + pred['VAL'][0] + '?'

In [0]:
#@title { display-mode: "form" }
def generate_questions(predictions):
  """Method that generates questions out of the given predicted triplets.
  :param predictions: array_like,
      list of triplets-dictionary of form 
      {'CONTEXT': [], 'OBJ': [], 'REL': [], 'VAL': []}
  :return questions: array_like,
      list of generated questions with context as answer
  :return questions_extended: array_like,
      list of more complex question forms 
      (multiple choice, True/False, Fill-the-gap)
  :return complex_objects: dict,
      dictionary containing object and substitutions as value 
      e.g. {"Christopher Douglas Adams" : "Vater von Douglas Adams}
    """
  questions = []
  questions_extended = []
  complex_objects = {}
  for i in range(len(predictions)):
    obj_cat = lookup_category(predictions[i]['OBJ'][0])
    val_cat = lookup_category(predictions[i]['VAL'][0])
    #print(predictions[i]['OBJ'], predictions[i]['REL'], predictions[i]['VAL'])
    rel = nlp(predictions[i]['REL'][0])
    rel_tags = [token.tag_ for token in rel]
    rel_tokens = [token.text for token in rel]
    rel_tags = ['XY' if rel_tags[i] in ['NN', 'NE'] and rel_tokens[i].islower() else rel_tags[i] for i in range(len(rel_tags))  ]
    #print(rel_tokens, rel_tags)
    if ('Zeitpunkt' in val_cat and 'am' not in predictions[i]['VAL'][0] and 'NN' not in rel_tags):
      value = 'am ' + predictions[i]['VAL'][0]
      predictions[i]['VAL'] = [value]
      if 'am' in rel_tokens:
        rel_tokens.remove('am')
        relation = ' '.join([t for t in rel_tokens])
        predictions[i]['REL'] = [relation]
        rel = nlp(predictions[i]['REL'][0])
    if ('NN' in rel_tags or 'NE' in rel_tags) and ('Zahl' in val_cat):
      #print(1)
      question = apply_rule1(predictions[i], rel, obj_cat, val_cat, is_template_a)
      questions.append((question, predictions[i]['CONTEXT'][0]))
      (true_stmt, false_stmt1, false_stmt2, gap_obj, gap_val, mc_obj, mc_val) = apply_rule1_extended(predictions[i], rel, obj_cat, val_cat, is_template_a)
      if true_stmt and false_stmt1 and false_stmt2:
        questions_extended.append((true_stmt, false_stmt1, false_stmt2, predictions[i]['CONTEXT'][0]))
        print(questions_extended[-1])
      if gap_obj:
        questions_extended.append((gap_obj, mc_obj, predictions[i]['CONTEXT'][0]))
        print(questions_extended[-1])
      if gap_val:
        questions_extended.append((gap_val, mc_val, predictions[i]['CONTEXT'][0]))
        print(questions_extended[-1])
      if mc_obj:
        questions_extended.append((question, mc_obj, predictions[i]['CONTEXT'][0]))
        print(questions_extended[-1])
    elif 'NN' in rel_tags or 'NE' in rel_tags:
      is_template_a = 'PTKVZ' in rel_tags or 'APPR' in rel_tags or 'APPRART' in rel_tags
      #print(2, is_template_a)
      question = apply_rule2(predictions[i], rel, obj_cat, val_cat, is_template_a)
      questions.append((question, predictions[i]['CONTEXT'][0]))
      (true_stmt, false_stmt1, false_stmt2, gap_obj, gap_val, mc_obj, mc_val) = apply_rule2_extended(predictions[i], rel, obj_cat, val_cat, is_template_a)
      if true_stmt and false_stmt1 and false_stmt2:
        questions_extended.append((true_stmt, false_stmt1, false_stmt2, predictions[i]['CONTEXT'][0]))
        print(questions_extended[-1])
      if gap_obj:
        questions_extended.append((gap_obj, mc_obj, predictions[i]['CONTEXT'][0]))
        print(questions_extended[-1])
      if gap_val:
        questions_extended.append((gap_val, mc_val, predictions[i]['CONTEXT'][0]))
        print(questions_extended[-1])
      if mc_obj:
        questions_extended.append((question, mc_obj, predictions[i]['CONTEXT'][0]))
        print(questions_extended[-1])
      #if is_template_a:
      aliases = [] if predictions[i]['OBJ'][0] not in complex_objects else complex_objects[predictions[i]['OBJ'][0]]
      aliases.append(predictions[i]['REL'][0] + ' ' + predictions[i]['VAL'][0])
      complex_objects[predictions[i]['OBJ'][0]] = list(set(aliases))
    elif 'VVPP' in rel_tags:
      is_template_a = 'PTKVZ' in rel_tags or 'APPR' in rel_tags or 'APPRART' in rel_tags
      #print(3, is_template_a)
      question = apply_rule3(predictions[i], rel, obj_cat, val_cat)
      questions.append((question, predictions[i]['CONTEXT'][0]))
      (true_stmt, false_stmt1, false_stmt2, gap_obj, gap_val, mc_obj, mc_val) = apply_rule3_extended(predictions[i], rel, obj_cat, val_cat)
      if true_stmt and false_stmt1 and false_stmt2:
        questions_extended.append((true_stmt, false_stmt1, false_stmt2, predictions[i]['CONTEXT'][0]))
        print(questions_extended[-1])
      if gap_obj:
        questions_extended.append((gap_obj, mc_obj, predictions[i]['CONTEXT'][0]))
        print(questions_extended[-1])
      if gap_val:
        questions_extended.append((gap_val, mc_val, predictions[i]['CONTEXT'][0]))
        print(questions_extended[-1])
      if mc_obj:
        questions_extended.append((question, mc_obj, predictions[i]['CONTEXT'][0]))
        print(questions_extended[-1])
    elif 'VVFIN' in rel_tags or 'VAFIN' in rel_tags:
      is_template_a = 'PTKVZ' in rel_tags
      #print(4, is_template_a)
      question = apply_rule4(predictions[i], rel, obj_cat, val_cat, is_template_a)
      questions.append((question, predictions[i]['CONTEXT'][0]))
      (true_stmt, false_stmt1, false_stmt2, gap_obj, gap_val, mc_obj, mc_val) = apply_rule4_extended(predictions[i], rel, obj_cat, val_cat, is_template_a)
      if true_stmt and false_stmt1 and false_stmt2:
        questions_extended.append((true_stmt, false_stmt1, false_stmt2, predictions[i]['CONTEXT'][0]))
        print(questions_extended[-1])
      if gap_obj:
        questions_extended.append((gap_obj, mc_obj, predictions[i]['CONTEXT'][0]))
        print(questions_extended[-1])
      if gap_val:
        questions_extended.append((gap_val, mc_val, predictions[i]['CONTEXT'][0]))
        print(questions_extended[-1])
      if mc_obj:
        questions_extended.append((question, mc_obj, predictions[i]['CONTEXT'][0]))
        print(questions_extended[-1])
    else:
      is_template_a = 'PTKVZ' in rel_tags
      #print(5, is_template_a)
      question = apply_rule5(predictions[i], rel, obj_cat, val_cat, is_template_a)
      questions.append((question, predictions[i]['CONTEXT'][0]))
      (true_stmt, false_stmt1, false_stmt2, gap_obj, gap_val, mc_obj, mc_val) = apply_rule5_extended(predictions[i], rel, obj_cat, val_cat, is_template_a)
      if true_stmt and false_stmt1 and false_stmt2:
        questions_extended.append((true_stmt + '\nTRUE', false_stmt1 + '\nFALSE', false_stmt2 + '\nFALSE', predictions[i]['CONTEXT'][0]))
        print(questions_extended[-1])
      if gap_obj:
        questions_extended.append((gap_obj, mc_obj, predictions[i]['CONTEXT'][0]))
        print(questions_extended[-1])
      if gap_val:
        questions_extended.append((gap_val, mc_val, predictions[i]['CONTEXT'][0]))
        print(questions_extended[-1])
      if mc_obj:
        questions_extended.append((question, mc_obj, predictions[i]['CONTEXT'][0]))
        print(questions_extended[-1])
    print(questions)
  return [questions, questions_extended, complex_objects]

In [0]:
#@title Frageset erstellen { display-mode: "form" }
#@markdown Generate question-set
def generate_question_set(predictions):
  """Wrapper around generate_questions with additional option 
    to generate complex questions.
    :param predictions: array_like,
      list of triplets-dictionary of form 
      {'CONTEXT': [], 'OBJ': [], 'REL': [], 'VAL': []}
    :return questions: array_like,
        list of generated questions with context as answer
    :return questions_extended: array_like,
        list of more complex question forms 
        (multiple choice, True/False, Fill-the-gap)
    :return complex_obj_questions: array_like,
        list of generated questions with complex objects """
  questions, questions_extended, complex_objects = generate_questions(predictions)
  complex_triplets = []
  for triplet in predictions:
    if triplet['OBJ'][0] in complex_objects and triplet['REL'][0] + ' ' + triplet['VAL'][0] not in complex_objects[triplet['OBJ'][0]]:
      for obj in complex_objects[triplet['OBJ'][0]]:
        value = triplet['VAL'][0].replace('am ', '')
        complex_triplets.append({'OBJ':[value],'REL':triplet['REL'],'VAL':[obj], 'CONTEXT': triplet['CONTEXT']})
  complex_obj_questions, b, c = generate_questions(complex_triplets)
  return [questions, questions_extended, complex_obj_questions]

baseline_questions, baseline_questions_extended, baseline_complex_obj_questions = generate_question_set(baseline_predictions)
print('\n')
questions, questions_extended, complex_obj_questions = generate_question_set(predictions)
clear_output()

In [0]:
#@title Fragen herunterladen (falls gewünscht)
#@markdown Download questions (if desired)
if 'download_questions' in globals() and download_questions:
  with open('baseline-question-set.txt', 'wt') as outfile:
    if baseline_questions:
      outfile.write('QUESTIONS\n')
      [outfile.write(q + '\n' + c + '\n') for q,c in baseline_questions]
    if baseline_questions_extended:
      outfile.write('\nEXTENDED QUESTIONS\n')
      for question in baseline_questions_extended:
        for q_item in question:
          if q_item:
            outfile.write(q_item + '\n')
        outfile.write('\n')
    if baseline_complex_obj_questions:
      outfile.write('\nCOMPLEX QUESTIONS\n')
      [outfile.write(q + '\n' + c + '\n') for q,c in baseline_complex_obj_questions]
  #files.download('baseline-question-set.txt') 
  with open('predictor-question-set.txt', 'wt') as outfile:
    if questions:
      outfile.write('QUESTIONS\n')
      [outfile.write(q + '\n' + c + '\n') for q,c in questions]
    if questions_extended:
      outfile.write('\nEXTENDED QUESTIONS\n')
      for question in questions_extended:
        for q_item in question:
          if q_item:
            outfile.write(q_item + '\n')
        outfile.write('\n')
    if complex_obj_questions:
      outfile.write('\nCOMPLEX QUESTIONS\n')
      [outfile.write(q + '\n' + c + '\n') for q, c in complex_obj_questions]
  #files.download('predictor-question-set.txt') 
else:
  if baseline_questions:
      print('QUESTIONS')
      [print(q + '\n' + c + '\n') for q,c in baseline_questions]
  if baseline_questions_extended:
    print('EXTENDED QUESTIONS')
    for question in baseline_questions_extended:
      for q_item in question:
        if q_item:
          print(q_item)
      print('\n')
  if baseline_complex_obj_questions:
    print('\nCOMPLEX QUESTIONS\n')
    [print(q + '\n' + c + '\n') for q,c in baseline_complex_obj_questions]
  #files.download('baseline-question-set.txt') 
  with open('predictor-question-set.txt', 'wt') as outfile:
    if questions:
      print('QUESTIONS\n')
      [print(q + '\n' + c + '\n') for q,c in questions]
    if questions_extended:
      print('\nEXTENDED QUESTIONS\n')
      for question in questions_extended:
        for q_item in question:
          if q_item:
            print(q_item + '\n')
        print('\n')
    if complex_obj_questions:
      print('\nCOMPLEX QUESTIONS\n')
      [print(q + '\n' + c + '\n') for q, c in complex_obj_questions]
if 'download_questions' in globals() and download_questions:
  files.download('baseline-question-set.txt')
  files.download('predictor-question-set.txt')

In [0]:
#@title { display-mode: "form" }
!rm baseline-question-set.txt
!rm predictor-question-set.txt
clear_output()